<a href="https://colab.research.google.com/github/singhsa3/Emotions/blob/main/librosaModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import GroupNorm, Linear, ReLU, GELU, CrossEntropyLoss, Sequential, Conv2d, Conv1d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout,AvgPool2d
from torch.optim import Adam, SGD
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision
from PIL import Image
import torchvision.transforms as T
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch


In [22]:
from google.colab import drive
drive.mount('/content/drive')

pathG='/content/drive/MyDrive/Pract/data'
cp = '/content/drive/MyDrive/Pretrained/wav2vec_small.pt'
sample="/content/drive/MyDrive/Pract/data/voice_samples/1173_GM1001_1326493712.wav"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import pickle
with open(pathG+"/featuresdict.pickle", "rb") as f:
  features=pickle.load(f)
with open(pathG+"/filenameslist.pickle", "rb") as f:
  fnames=pickle.load(f)
import pandas as pd
df=pd.read_csv(pathG+"/labels/Yared Alemu_fear.csv")
df = df[(df['name'].isin (features.keys()))].reset_index()
df['name2'] =df['name'].apply (lambda x: x.split(".")[0]+".png")
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(df['name2'], df['emotion'], test_size = 0.3)

In [24]:
import shutil
import os
from os.path import exists
import os

try:
  os.makedirs( pathG+"/temp/train/n")
  os.makedirs( pathG+"/temp/train/y")
  os.makedirs( pathG+"/temp/test/n")
  os.makedirs( pathG+"/temp/test/y")
except:
  print("directories exist")

import zipfile
with zipfile.ZipFile(pathG+"/greyimg.zip", 'r') as zip_ref:
    zip_ref.extractall(pathG+"/"+'greyimg')

for src in list(train_x):
  
  src2=pathG+"/greyimg/"+src
  lbl = df[df["name2"]==src]['emotion'].values[0]
  if lbl==0 :
    dst= pathG+"/temp/train/n/"+src
  else:
    dst= pathG+"/temp/train/y/"+src
  if  exists(dst)==False:
    shutil.copyfile(src2, dst)

for src in list(val_x):
  lbl = df[df["name2"]==src]['emotion'].values[0]
  src2=pathG+"/greyimg/"+src
  if lbl==0 :
    dst= pathG+"/temp/test/n/"+src
  else:
    dst= pathG+"/temp/test/y/"+src
  if  exists(dst)==False:
    shutil.copyfile(src2, dst)

  

directories exist


In [25]:
def get_data():
    data_dir = pathG+"/temp"
    print(data_dir)
   
    transform = transforms.Compose([
        #transforms.RandomRotation(20),
        #transforms.RandomResizedCrop(128),
        #transforms.RandomHorizontalFlip(),
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor()])

    train_set = datasets.ImageFolder(data_dir + '/train', transform=transform)
    test_set = datasets.ImageFolder(data_dir + '/test', transform=transform)

    train = DataLoader(train_set, batch_size=8, shuffle=True)
    test = DataLoader(test_set, batch_size=8, shuffle=True)

    return train, test

In [26]:
def train_imshow():
    classes = (0, 1) # Defining the classes we have
    dataiter = iter(train)
    images, labels = dataiter.next()
    fig, axes = plt.subplots(figsize=(10, 4), ncols=5)
    for i in range(5):
        ax = axes[i]
        ax.imshow(images[i].permute(1, 2, 0)) 
        ax.title.set_text(' '.join('%5s' % classes[labels[i]]))
    plt.show()

In [27]:
train, test = get_data()


/content/drive/MyDrive/Pract/data/temp


In [28]:
class Net(nn.Module):   
    def __init__(self):
        super(Net, self).__init__()
        
        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=0), 
            nn.ReLU(),    
            nn.AvgPool2d(kernel_size=2, stride=1), 

            nn.Conv2d(8, 16, kernel_size=2, stride=1, padding=0),
            nn.ReLU(),                       
            AvgPool2d(kernel_size=3, stride=1),

            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            AvgPool2d(kernel_size=2, stride=1),         
            
        )
         
        self.linear_layers = Sequential(
            nn.Linear(1479200,25088),
            nn.ReLU(),
            nn.Linear(25088, 2048),
            nn.ReLU(),            
            nn.Linear(2048, 2048),
            nn.ReLU(),
            nn.Linear(2048, 2),           
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [ ]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

if torch.cuda.is_available():
    model = net.cuda()
    criterion = criterion.cuda()


def train_net(n_epoch):
    losses = []
    for epoch in range(n_epoch):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(train, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            losses.append(loss)
            running_loss += loss.item()
            if i % 100 == 99:  # print every 2000 mini-batches
                print('[%d, %5d] loss: %.10f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0
    plt.plot(losses, label='Training loss')
    plt.show()
    print('Finished Training')

train_net(10)

#PATH = './cat_dog_net.pth'
# torch.save(net.state_dict(), PATH)

# Loading the trained network
# net.load_state_dict(torch.load(PATH))

correct = 0
total = 0
with torch.no_grad():
    for data in test:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the %d test images: %d %%' % (len(test),
    100 * correct / total))

_______________________________________________
## @REECE, Do NOT USE ANYTHING BELOW THIS

In [ ]:
data_dir = pathG+"/temp"
# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [ ]:
x=features[df.iloc[0][3]]
x= np.pad(x, ((10,10),(10, 6500-x.shape[1])), 'constant')
arr =np.expand_dims(x, axis=0)
#print(arr.shape)
for i,row in df.iterrows():
  if i>0:
    ft = features[row['name']]
    ft= np.pad(ft, ((10,10),(10, 6500-ft.shape[1])), 'constant')
    arr2 = np.expand_dims(ft, axis=0)
    #print(arr2.shape)
    arr = np.vstack((arr,arr2))
labels=np.array(df.emotion)

In [ ]:
import pickle
with open(pathG+"/curfile.pickle", "wb") as f:
  pickle.dump((arr, labels),f)

In [ ]:
# Starting point

In [ ]:
# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import GroupNorm, Linear, ReLU, GELU, CrossEntropyLoss, Sequential, Conv2d, Conv1d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout,AvgPool2d, Sigmoid
from torch.optim import Adam, SGD
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
class Fp32GroupNorm(nn.GroupNorm):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def forward(self, input):
        output = F.group_norm(
            input.float(),
            self.num_groups,
            self.weight.float() if self.weight is not None else None,
            self.bias.float() if self.bias is not None else None,
            self.eps,
        )
        return output.type_as(input)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
pathG='/content/drive/MyDrive/Pract/data'
cp = '/content/drive/MyDrive/Pretrained/wav2vec_small.pt'
sample="/content/drive/MyDrive/Pract/data/voice_samples/1173_GM1001_1326493712.wav"

In [ ]:
import pickle
with open(pathG+"/curfile.pickle", "rb") as f:
  abc = pickle.load(f)
arr=abc[0]
labels=abc[1]

In [ ]:
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(arr, labels, test_size = 0.3)

In [ ]:
# converting training images into torch format
train_x = train_x.reshape(train_x.shape[0], 1, train_x.shape[1], train_x.shape[2])
train_x  = torch.from_numpy(train_x)

# converting the target into torch format
train_y = train_y.astype(int);
train_y = torch.from_numpy(train_y)

# shape of training data
train_x.shape, train_y.shape
# converting validation images into torch format
val_x = val_x.reshape(val_x.shape[0], 1, val_x.shape[1], val_x.shape[2])
val_x  = torch.from_numpy(val_x)

# converting the target into torch format
val_y = val_y.astype(int);
val_y = torch.from_numpy(val_y)

# shape of validation data
val_x.shape, val_y.shape

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
xbbbxbbxbxb

In [ ]:
class Net(nn.Module):   
    def __init__(self):
        super(Net, self).__init__()
        n= 808 # 240796 #117072 #6496 #14616 #
        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            nn.Conv2d(1, 4, kernel_size=2, stride=2, padding=0),
            #BatchNorm2d(4),
            ReLU(),
            MaxPool2d(kernel_size=2, stride=2), 
            nn.Conv2d(4, 4, kernel_size=2, stride=2, padding=0),
            #BatchNorm2d(4),
            ReLU(),
            MaxPool2d(kernel_size=2, stride=2),  
            nn.Conv2d(4, 4, kernel_size=2, stride=2, padding=0),
            #BatchNorm2d(4),
            ReLU(),
            MaxPool2d(kernel_size=2, stride=2),        
       
            
        )
         
        self.linear_layers = Sequential(
            Linear(n, int(n/2)),
            ReLU(),
            Linear(int(n/2), int(n/4)),
            ReLU(),
            Linear(int(n/4), 2),
            #Sigmoid()
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.1)
# defining the loss function
criterion = nn.CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
#print(model)
# Define the batch size and the number of epochs
BATCH_SIZE = 10
N_EPOCHS = 100
torch.cuda.empty_cache()
# Use torch.utils.data to create a DataLoader 
# that will take care of creating batches 
dataset = TensorDataset(train_x, train_y)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

# Get the dataset size for printing (it is equal to N_SAMPLES)
dataset_size = len(dataloader.dataset)
# empty list to store training losses
train_losses = [10000000,1000000,1000000,10000000,1000000]
# empty list to store validation losses
val_losses = []

# Loop over epochs
for epoch in range(N_EPOCHS):
    mnl = np.mean(train_losses)
    print(f"Epoch {epoch + 1}\n-------------------------------")

    # Loop over batches in an epoch using DataLoader
    for id_batch, (x_batch, y_batch) in enumerate(dataloader):
      if torch.cuda.is_available():
        x_batch = x_batch.cuda()
        y_batch = y_batch.cuda()
        #x_val = val_x.cuda()
        #y_val = val_y.cuda()

        y_batch_pred = model(x_batch)
        #output_val = model(x_val)

        loss = criterion(y_batch_pred, y_batch)
        #loss_val = criterion(output_val, y_val)
        #train_losses.append(loss)
        #val_losses.append(loss_val)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Every 100 batches, print the loss for this batch
        # as well as the number of examples processed so far 
        if id_batch % 100 == 0:
            loss, current = loss.item(), (id_batch + 1)* len(x_batch)
            print(f"loss: {loss:>7f}  [{current:>5d}/{dataset_size:>5d}]")    
    train_losses.pop()
    train_losses.insert(0,loss.detach().cpu().numpy())
    #if np.mean(train_losses)>mnl:
      #break
torch.cuda.empty_cache()

from sklearn.metrics import accuracy_score
# prediction for validation set
predictions=[]
dataset = TensorDataset(val_x, val_y)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
for id_batch, (x_batch, y_batch) in enumerate(dataloader):
      if torch.cuda.is_available():
        x_batch = x_batch.cuda()
        y_batch = y_batch.cuda()
        x_val = val_x.cuda()
        y_val = val_y.cuda()
        output = model(x_batch)
        softmax = torch.exp(output).cpu()
        prob = list(softmax.detach().cpu().numpy())
        prediction = np.argmax(prob, axis=1)
        predictions = predictions+list(prediction)

        

# accuracy on validation set
accuracy_score(val_y, np.array(predictions))

In [ ]:
from sklearn.metrics import accuracy_score
# prediction for validation set
predictions=[]
dataset = TensorDataset(train_x, train_y)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)
for id_batch, (x_batch, y_batch) in enumerate(dataloader):
      if torch.cuda.is_available():
        x_batch = x_batch.cuda()
        y_batch = y_batch.cuda()
        x_val = train_x.cuda()
        y_val = train_y.cuda()
        output = model(x_batch)
        softmax = torch.exp(output).cpu()
        prob = list(softmax.detach().cpu().numpy())
        prediction = np.argmax(prob, axis=1)
        predictions = predictions+list(prediction)

        

# accuracy on validation set
accuracy_score(train_y, np.array(predictions))

In [ ]:
 # Defining another 2D convolution layer
            Conv2d(2, 4, kernel_size=2, stride=1, padding=0),
            #BatchNorm2d(16),
            ReLU(),
            AvgPool2d(kernel_size=4, stride=1),
            # Defining another 2D convolution layer
            Conv2d(4, 4, kernel_size=2, stride=1, padding=0),
            #BatchNorm2d(64),
            ReLU(),
            MaxPool2d(kernel_size=4, stride=1),

In [ ]:
torch.save(model, 'model.bin')

In [ ]:
model = torch.load('model.bin')
model.eval()

In [ ]:
import torch
torch.cuda.empty_cache()
!nvidia-smi
a = torch.cuda.memory_allocated(0)
c = torch.cuda.memory_reserved(0)
print(a,c)